In [1]:
import os
import warnings

warnings.simplefilter(action="ignore")
os.environ["GRPC_VERBOSITY"] = "NONE"

# Prerequisites

Please make sure your environmental variables and dependencies are ready to use LLM services.

In [2]:
from dotenv import load_dotenv

load_dotenv("../../.env_api")

True

# How to input langrila's modules

In langrila, universal message system allows you to handle each client message in a same way.

# Import modules

In [3]:
from langrila import (
    InMemoryConversationMemory,
    ToolConfig,
    ToolParameter,
    ToolProperty,
)
from langrila.claude import ClaudeFunctionalChat
from langrila.gemini import GeminiFunctionalChat
from langrila.openai import OpenAIFunctionalChat

# Define tools

In this example, we can use these tools.

In [4]:
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    return f"Disco ball is {'spinning!' if power else 'stopped.'}"


def start_music(energetic: bool, loud: bool, bpm: str) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The music property.
    """
    return f"Starting music! {energetic=} {loud=}, {bpm=}"


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    return f"Lights are now set to {brightness}"

# Tool definition

In langrila, you can define tool configs via ToolProperty, ToolParameter and ToolConfig object.

In [5]:
tool_configs = [
    ToolConfig(
        name="power_disco_ball",
        description="Powers the spinning disco ball.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="power",
                    type="boolean",
                    description="Boolean to spin disco ball.",
                ),
            ],
            required=["power"],
        ),
    ),
    ToolConfig(
        name="start_music",
        description="Play some music matching the specified parameters.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="energetic",
                    type="boolean",
                    description="Whether the music is energetic or not.",
                ),
                ToolProperty(
                    name="loud", type="boolean", description="Whether the music is loud or not."
                ),
                ToolProperty(
                    name="bpm",
                    type="string",
                    description="The beats per minute of the music.",
                    enum=["60", "120", "180"],  # Gemini doesn't support numerical enum
                ),
            ],
            required=["energetic", "loud", "bpm"],
        ),
    ),
    ToolConfig(
        name="dim_lights",
        description="Dim the lights.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="brightness",
                    type="number",
                    description="The brightness of the lights, 0.0 is off, 1.0 is full.",
                ),
            ],
            required=["brightness"],
        ),
    ),
]

In [6]:
prompt = "Turn this place into a comfortable party mood!"

### OpenAI Chat Completion

ToolConfig object is internally converted to client tool configs

In [7]:
chat_openai = OpenAIFunctionalChat(api_key_env_name="API_KEY")

In [8]:
response = chat_openai.run(
    prompt,
    model_name="gpt-4o-2024-08-06",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

# Response is CompletionResults object
print(response.message.content[0].text)

The place is now set for a party mood! The disco ball is spinning, energetic music has started playing at 120 BPM, and the lights are dimmed to create the perfect ambiance. Time to enjoy the party! 🪩🎶💃


In [9]:
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The place is now set for a party mood! The disco ball is spinning, energetic music has started playing at 120 BPM, and the lights are dimmed to create the perfect ambiance. Time to enjoy the party! \U0001faa9🎶💃'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-2024-08-06',
  'prompt_tokens': 305,
  'completion_tokens': 126},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'audio': None,
   'function_call': None,
   'tool_calls': [{'id': 'call_QtkgH0uYKKcZ54YeVvtVMFWi',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_3wXldM13FWdA5Eer9X7bGFNP',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': '

Or you can also pass tools when initializing.

In [10]:
chat_openai_tool_only = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-2024-08-06",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [11]:
prompt = "Turn this place into a comfortable party mood!"
response = chat_openai_tool_only.run(prompt)
print(response.message.content[0].text)

The place is now set for a comfortable party mood! The disco ball is spinning, the music is energetic yet not too loud, and the lights are dimmed to create a cozy atmosphere. Enjoy the party!


Usage in the response object is total usage of the usage to call tool and the usage to use tool.

In [12]:
response.usage

Usage(prompt_tokens=305, completion_tokens=117, total_tokens=422)

Asynchronous generation

In [13]:
response = await chat_openai.arun(
    prompt,
    model_name="gpt-4o-2024-08-06",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)
response

CompletionResults(message=Message(role='assistant', content=[TextContent(text='The place is now ready for a party vibe! The disco ball is spinning, lively music is playing at 120 BPM, and the lights are dimmed for the perfect ambiance. Enjoy the party!')], name=None), usage=Usage(prompt_tokens=305, completion_tokens=115, total_tokens=420), prompt=[{'role': 'user', 'content': [{'type': 'text', 'text': 'Turn this place into a comfortable party mood!'}], 'name': 'User'}, {'content': None, 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': [{'id': 'call_F6TMj9mmKv4jyBqGQ4hvHca3', 'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'}, 'type': 'function'}, {'id': 'call_W1KTwoH1e8KEcDYYPXGfmEnQ', 'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}', 'name': 'start_music'}, 'type': 'function'}, {'id': 'call_gjn4021uitIRl3bfiEAYjwep', 'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'}, 'type': '

multiple generation

In [14]:
response = chat_openai.run(
    prompt,
    n_results=2,
    model_name="gpt-4o-2024-08-06",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': 'The place is now in a comfortable party mood with the disco ball spinning, energizing music playing at 120 BPM, and the lights dimmed to a cozy brightness. Let the party begin!'},
  {'text': 'The place is now set for a comfortable party mood. The disco ball is spinning, music with an energetic beat at 120 BPM is playing loudly, and the lights are dimmed to set the perfect ambiance. Enjoy the party!'}],
 'name': None}

You can get the results from specific tool.

In [15]:
response = chat_openai.run(
    prompt,
    tool_choice="start_music",
    model_name="gpt-4o-2024-08-06",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

# Show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "I've started some energetic and loud music at 120 BPM to get the party mood going! If you need further suggestions, like lighting or decor, just let me know."}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-2024-08-06',
  'prompt_tokens': 244,
  'completion_tokens': 51},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'audio': None,
   'function_call': None,
   'tool_calls': [{'id': 'call_mbxIp4dNnomrdH2dvBnaINVZ',
     'function': {'arguments': '{"energetic":true,"loud":true,"bpm":"120"}',
      'name': 'start_music'},
     'type': 'function'}]},
  {'role': 'tool',
   'tool_call_id': 'call_mbxIp4dNnomrdH2dvBnaINVZ',
   'name': 'start_music',
   'content': "Starting music! energetic=True loud=True, bpm='120'"}]}

If you only want to get the results of the tools, please specify tool_only option as True, then you get the FunctionCallingResults object instead of CompletionResults object.

In [16]:
response = chat_openai.run(
    prompt,
    tool_only=True,  # You can also specify in iniitalization
    model_name="gpt-4o-2024-08-06",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)
response

FunctionCallingResults(usage=Usage(prompt_tokens=171, completion_tokens=75, total_tokens=246), results=[Message(role='function', content=[ToolContent(output='Disco ball is spinning!', call_id='call_MUMKQtoFwIrtcQp9fHiRSuxQ', args='{"power": true}', funcname='power_disco_ball')], name='power_disco_ball'), Message(role='function', content=[ToolContent(output="Starting music! energetic=True loud=True, bpm='120'", call_id='call_59fuLF0NOYZmCZxVvlVJILdw', args='{"energetic": true, "loud": true, "bpm": "120"}', funcname='start_music')], name='start_music'), Message(role='function', content=[ToolContent(output='Lights are now set to 0.5', call_id='call_NxVPfw9ZqGc60MwVBRHcZGUh', args='{"brightness": 0.5}', funcname='dim_lights')], name='dim_lights')], calls=Message(role='function_call', content=[ToolCall(name='power_disco_ball', args='{"power": true}', call_id='call_MUMKQtoFwIrtcQp9fHiRSuxQ'), ToolCall(name='start_music', args='{"energetic": true, "loud": true, "bpm": "120"}', call_id='call_5

FunctionCallingResults instance includes the output of the function.

In [17]:
response.results

[Message(role='function', content=[ToolContent(output='Disco ball is spinning!', call_id='call_MUMKQtoFwIrtcQp9fHiRSuxQ', args='{"power": true}', funcname='power_disco_ball')], name='power_disco_ball'),
 Message(role='function', content=[ToolContent(output="Starting music! energetic=True loud=True, bpm='120'", call_id='call_59fuLF0NOYZmCZxVvlVJILdw', args='{"energetic": true, "loud": true, "bpm": "120"}', funcname='start_music')], name='start_music'),
 Message(role='function', content=[ToolContent(output='Lights are now set to 0.5', call_id='call_NxVPfw9ZqGc60MwVBRHcZGUh', args='{"brightness": 0.5}', funcname='dim_lights')], name='dim_lights')]

In [18]:
# Tool call messages

response.calls

Message(role='function_call', content=[ToolCall(name='power_disco_ball', args='{"power": true}', call_id='call_MUMKQtoFwIrtcQp9fHiRSuxQ'), ToolCall(name='start_music', args='{"energetic": true, "loud": true, "bpm": "120"}', call_id='call_59fuLF0NOYZmCZxVvlVJILdw'), ToolCall(name='dim_lights', args='{"brightness": 0.5}', call_id='call_NxVPfw9ZqGc60MwVBRHcZGUh')], name=None)

Streaming

In [19]:
response = chat_openai.stream(
    prompt,
    model_name="gpt-4o-2024-08-06",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

# Response is CompletionResults object
[r for r in response]

[CompletionResults(message=Message(role='assistant', content=[TextContent(text='')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}], raw=None),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}], raw=None),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The place')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}], raw=None),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The place is')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}], raw=None),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The place is now')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}], raw=None),
 CompletionResults(m

In [20]:
response = chat_openai.astream(
    prompt,
    model_name="gpt-4o-2024-08-06",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

# Response is CompletionResults object
[r async for r in response]

[CompletionResults(message=Message(role='assistant', content=[TextContent(text='')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}], raw=None),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}], raw=None),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The place')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}], raw=None),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The place is')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}], raw=None),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The place is now')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}], raw=None),
 CompletionResults(m

# Pydantic tools

You can define tools as pydantic models. To use pydantic model as tool, make sure the following points: 

- Write description of top level docstring field in the class. This is used for tool description.
- Define each argument fileds with description
- Implement run() method.

Tool definition using `ToolProperty`, `ToolParameter` and `ToolConfig` is almost equivalent to the following way. 

In [21]:
from enum import Enum
from typing import Literal

from pydantic import BaseModel, Field


class PowerDiscoBall(BaseModel):
    """Powers the spinning disco ball."""

    power: bool = Field(description="Boolean to spin disco ball.")

    def run(self) -> str:
        return f"Disco ball is {'spinning!' if self.power else 'stopped.'}"


class DimLights(BaseModel):
    """Dim the lights."""

    brightness: float = Field(description="The brightness of the lights, 0.0 is off, 1.0 is full.")

    def run(self) -> str:
        return f"Lights are now set to {self.brightness}"


class BPM(Enum):
    """The beats per minute of the music."""

    SLOW = "60"
    MEDIUM = "120"
    FAST = "180"


class StartMusic(BaseModel):
    """Play some music matching the specified parameters."""

    energetic: bool = Field(description="Whether the music is energetic or not.")
    loud: bool = Field(description="Whether the music is loud or not.")
    bpm: BPM = Field(description="The beats per minute of the music.")
    # bpm: Literal["60", "120", "180"] = Field(..., description="The beats per minute of the music.") # This also works

    def run(self) -> str:
        return f"Starting music! {self.energetic=} {self.loud=}, {self.bpm=}"

In [22]:
# Convert to ToolConfig

tools_pydantic = [PowerDiscoBall, StartMusic, DimLights]
tool_configs_pydantic = [ToolConfig.from_pydantic(tool) for tool in tools_pydantic]

In [23]:
chat_openai = OpenAIFunctionalChat(api_key_env_name="API_KEY")

In [24]:
response = chat_openai.run(
    prompt,
    model_name="gpt-4o-mini-2024-07-18",
    tools=tools_pydantic,
    tool_configs=tool_configs_pydantic,
)

# Response is CompletionResults object
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The party mood is now in full swing! 🎉 The disco ball is spinning, energetic music is playing loudly, and the lights are dimmed for that perfect atmosphere. Time to dance and have fun! 🕺💃'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 320,
  'completion_tokens': 121},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'audio': None,
   'function_call': None,
   'tool_calls': [{'id': 'call_OgONBX5hrbH8HYFBQtW7kZ2q',
     'function': {'arguments': '{"power": true}', 'name': 'PowerDiscoBall'},
     'type': 'function'},
    {'id': 'call_m3HIu4y5NW1tsapEkK7kjvxW',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'StartMusic'},
     'type': 'function'},
    {'id': 'call_OaDP8bJ32XBjeDcl

### Array property for OpenAI API

Function Calling with array property for OpenAI Chat Completion.

In [25]:
def get_ingredients(ingredients: list[str]) -> str:
    """Prepare ingredients to make a meal."""
    return f"Preparing {ingredients}."


tool_configs_array = [
    ToolConfig(
        name="get_ingredients",
        description="Prepare ingredients to make a meal.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="ingredients",
                    type="array",
                    description="List of ingredients.",
                    items={"type": "string"},
                ),
            ],
            required=["ingredients"],
        ),
    ),
]

In [26]:
chat_openai = OpenAIFunctionalChat(api_key_env_name="API_KEY")

In [27]:
response = chat_openai.run(
    prompt="What are the ingredients to make omelet rice?",
    tool_choice="get_ingredients",
    model_name="gpt-4o-2024-08-06",
    tools=[get_ingredients],
    tool_configs=tool_configs_array,
)
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "To make omelet rice, you'll need the following ingredients:\n\n- Rice\n- Eggs\n- Onion\n- Green peas\n- Carrots\n- Chicken breast\n- Soy sauce\n- Salt\n- Black pepper\n- Ketchup\n- Butter or oil for cooking\n\nThese ingredients will help you prepare the classic dish known as omurice, combining savory fried rice with a fluffy omelet."}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-2024-08-06',
  'prompt_tokens': 194,
  'completion_tokens': 118},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'What are the ingredients to make omelet rice?'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'audio': None,
   'function_call': None,
   'tool_calls': [{'id': 'call_b2egSX3R23uApcZKxtZfxDdB',
     'function': {'arguments': '{"ingredients":["rice","eggs","onion","green peas","carrots","chicken breast","soy sauce","salt","black pepper","ketchup","butter or oil"]}',
  

### Azure OpenAI Chat Completion 

In [28]:
chat_openai_azure = OpenAIFunctionalChat(
    api_key_env_name="AZURE_API_KEY",
    api_type="azure",
    api_version="2024-05-01-preview",
    endpoint_env_name="AZURE_ENDPOINT",
    deployment_id_env_name="AZURE_DEPLOYMENT_ID",
)

In [29]:
response = chat_openai_azure.run(
    prompt,
    model_name="gpt-4o-2024-05-13",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "The disco ball is spinning, the energetic music is pumping at 120 bpm, and the lights are dimmed to create a perfect party ambiance. Let's get this party started!"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-2024-05-13',
  'prompt_tokens': 305,
  'completion_tokens': 110},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'audio': None,
   'function_call': None,
   'tool_calls': [{'id': 'call_7xwTGw3hrzJIJECffhqxpmlA',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_qWkwJ3hHakU6t6fl11STQbmg',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_v4ESReEribrSBH6QqFaIVl5z',
     'function':

In [30]:
response = await chat_openai_azure.arun(
    prompt,
    model_name="gpt-4o-2024-05-13",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'Alright! The disco ball is spinning, the energetic music is blasting at 120 BPM, and the lights are dimmed to a cozy 50%. The party mood is set! 🕺💃'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-2024-05-13',
  'prompt_tokens': 305,
  'completion_tokens': 116},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'audio': None,
   'function_call': None,
   'tool_calls': [{'id': 'call_L7V2snfpNg7gB6vbn8sHLcVO',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_vKCKRpJmuzWwOrHwJ796K1xZ',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_ZDlqyHA7AgRZFBcFmn4JaWWO',
     'function': {'arguments': 

In [31]:
response = chat_openai_azure.run(
    prompt,
    tool_choice="start_music",
    model_name="gpt-4o-2024-05-13",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "The music has started! Let's add a few more elements to ramp up the party:\n\n1. **Lighting**: Set up some colorful LED lights or fairy lights. If you have smart lights, adjust them to dynamic, colorful modes that sync with the music.\n2. **Seating and Space**: Arrange comfortable seating areas but also create an open space for dancing.\n3. **Decorations**: Hang some fun, festive decorations such as streamers, balloons, or banners.\n4. **Refreshments**: Set up a snack and drink station with a variety of options. Consider themed cups and plates to keep it festive.\n5. **Games and Activities**: Prepare some party games or activities like a photo booth with props, dance-off area, or even a karaoke machine if that's your style.\n\nNow you've got a comfortable yet energetic party atmosphere!"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-2024-05-13',
  'prompt_tokens': 244,
  'completion_tokens': 189},
 'prompt': [{'role': 'us

In [32]:
response = chat_openai_azure.stream(
    prompt,
    model_name="gpt-4o-2024-05-13",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)
[r for r in response]

[CompletionResults(message=Message(role='assistant', content=[TextContent(text='')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}], raw=None),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}], raw=None),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The lights')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}], raw=None),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The lights are')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}], raw=None),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The lights are dim')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}], raw=None),
 CompletionResu

In [33]:
response = chat_openai_azure.astream(
    prompt,
    model_name="gpt-4o-2024-05-13",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)
[r async for r in response]

[CompletionResults(message=Message(role='assistant', content=[TextContent(text='')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}], raw=None),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}], raw=None),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The disco')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}], raw=None),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The disco ball')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}], raw=None),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The disco ball is')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}], raw=None),
 CompletionResult

### Gemini on Google Generative AI

You can use tools for Gemini as well which defined for OpenAI Chat Completion.

In [34]:
gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
)

In [35]:
response = gemini.run(
    prompt,
    # tool_only=True,
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "The lights are dimmed, upbeat music is playing, and the disco ball is spinning! Let's get this party started! 🎉 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 353,
  'completion_tokens': 84},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "dim_lights"
      args {
        fields {
          key: "brightness"
          value {
            number_value: 0.5
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }
        }
      }
   

Asynchronous generation

In [36]:
response = await gemini.arun(
    prompt,
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, I've dimmed the lights, turned on the disco ball, and started some upbeat music!  Let's get this party started! 🎉 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 353,
  'completion_tokens': 89},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "dim_lights"
      args {
        fields {
          key: "brightness"
          value {
            number_value: 0.5
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
     

Gemini allows you to call specific tool as well as OpenAI Chat Completion.

In [37]:
response = gemini.run(
    prompt,
    tool_choice="start_music",
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Okay, I'm setting the mood!  🎶 \n\nLet's get this party started! 🎉  What else can I do to make it awesome?  Need some lighting ideas? 😉 💡 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 276,
  'completion_tokens': 67},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }
        }
      }
    }
  }
  role: "model",
  parts {
    function_response {
      name: "start_music"
      response {
        fields {
          key: "content"
          value {
            st

Also pydantic tools is acceptable.

In [38]:
gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-pro",
    tools=tools_pydantic,
    tool_configs=tool_configs_pydantic,
)

In [39]:
response = gemini.run(prompt)
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, I've got the disco ball spinning, the music bumping, and the lights dimmed! Let's get this party started! 🎉 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 353,
  'completion_tokens': 83},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "PowerDiscoBall"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "StartMusic"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }
        }
      }
 

Streaming

In [40]:
response = gemini.stream(
    prompt,
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)
[r for r in response]

[CompletionResults(message=Message(role='assistant', content=[TextContent(text='Alright')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt='', raw=None),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="Alright, I've dimmed the lights, turned on the disco ball, and started")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt='', raw=None),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="Alright, I've dimmed the lights, turned on the disco ball, and started some energetic music!  Let's get this party started! What else can")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt='', raw=None),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="Alright, I've dimmed the lights, turned on the disco ball, and started some energetic music!  Let's get this party started! What else can I do to se

In [41]:
response = gemini.astream(
    prompt,
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)
[r async for r in response]

[CompletionResults(message=Message(role='assistant', content=[TextContent(text='The')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt='', raw=None),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The lights are dimmed, the disco ball is spinning, and the music is pumping!')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt='', raw=None),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="The lights are dimmed, the disco ball is spinning, and the music is pumping! Let's get this party started! 🎉 \n")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt='', raw=None),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="The lights are dimmed, the disco ball is spinning, and the music is pumping! Let's get this party started! 🎉 \n")], name=None), usage=Usage(prompt_tokens=353, completion_to

### Gemini on VertexAI

In [42]:
gemini_vertexai = GeminiFunctionalChat(
    api_type="vertexai",
    project_id_env_name="PROJECT_ID",
    location_env_name="LOCATION",
)

In [43]:
response = gemini_vertexai.run(
    prompt=prompt,
    model_name="gemini-1.5-flash",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Okay, I've got this!  Here's how we can create a comfortable party mood:\n\n**Music:**\n\n* **Upbeat but not overwhelming:** I've started some music with a BPM of 120.  This is a nice, energetic tempo that gets people moving but isn't too fast. \n* **Keep it chill:** I've made sure the music isn't too loud.  You want people to be able to chat and mingle without having to shout over the music.\n\n**Lighting:**\n\n* **Dim the lights:**  I've dimmed the lights to create a cozy and inviting atmosphere.  \n* **Disco ball optional:** I've turned on the disco ball for a little extra fun.  You can adjust this if you prefer a more subdued vibe.\n\n**Atmosphere:**\n\n* **Set the mood with scents:**  Consider lighting some candles or diffusing essential oils like lavender or bergamot to create a relaxing and inviting atmosphere.\n* **Comfortable seating:** Make sure there are plenty of comfortable places for people to sit and relax. \n* **

multiple generation

In [44]:
response = gemini_vertexai.run(
    prompt=prompt,
    n_results=2,
    model_name="gemini-1.5-flash",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': 'Okay, I\'ve got the music pumping, the lights dimmed for a cozy vibe, and the disco ball sparkling!  Let\'s add some more party touches: \n\n* **"Let\'s get this party started!"** I\'ll announce it over the speakers. \n* **"Want to play some party games?"**  I\'ll suggest some fun options. \n* **"Who\'s ready for some snacks?"**  I\'ll encourage people to grab some food and drinks.\n\nDo you want me to do anything else to create a comfortable party mood?  \n'},
  {'text': "Okay, I'm setting the mood! \n\nI've started some upbeat music with a medium tempo. The lights are dimmed to a cozy level, and the disco ball is spinning, casting shimmering light around the room.  \n\nIs there anything else I can do to make this party more comfortable and fun?  Tell me about your ideal party atmosphere - do you want it to be more energetic, or more relaxed?  Do you need any snacks or drinks?  Let me know! \n"}],
 'name': None}

Pydantic tool use

In [45]:
gemini_vertexai = GeminiFunctionalChat(
    api_type="vertexai",
    project_id_env_name="PROJECT_ID",
    location_env_name="LOCATION",
)

In [46]:
response = gemini_vertexai.run(
    prompt=prompt,
    model_name="gemini-1.5-flash",
    tools=tools_pydantic,
    tool_configs=tool_configs_pydantic,
)
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Okay, I'm turning up the party vibe!  \n\n* **Music is playing!**  It's got a good beat, not too loud, perfect for a chill but fun atmosphere. \n* **Lights are dimmed** to create a cozy, inviting feeling. \n* **The disco ball is spinning!** Adding a touch of sparkle and fun. \n\nLet's get this party started!  What else can I do to make it even better?  Do you have any specific requests for drinks, decorations, or games? 🎉 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-flash',
  'prompt_tokens': 181,
  'completion_tokens': 132},
 'prompt': [role: "user"
  parts {
    text: "Turn this place into a comfortable party mood!"
  },
  role: "model"
  parts {
    function_call {
      name: "StartMusic"
      args {
        fields {
          key: "loud"
          value {
            bool_value: false
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }

### Anthropic Claude

tool_configs can be shared with Claude.

In [47]:
claude = ClaudeFunctionalChat(api_key_env_name="ANTHROPIC_API_KEY")

In [48]:
response = claude.run(
    prompt=prompt,
    model_name="claude-3-5-sonnet-20240620",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up a comfortable party mood for you. Here's what I've done:\n\n1. Dimmed the lights: I've set the brightness to 0.6 (60% brightness). This creates a cozy atmosphere without making it too dark.\n\n2. Activated the disco ball: The spinning disco ball will add some fun, party-like lighting effects to the room.\n\n3. Started the music: I've chosen energetic music to keep the party mood up, but set it to not be too loud so people can still comfortably chat. The tempo is set to 120 BPM, which is a good, upbeat pace for a party without being too intense.\n\nThis combination should create a comfortable party atmosphere that's lively but not overwhelming. The dimmed lights and spinning disco ball will create a fun, party-like ambiance, while the energetic but not-too-loud music will keep the mood up without hindering conversation.\n\nIs there anything you'd like to adjust? Perhaps make the music louder, change the tempo, 

Asynchronous generation

In [49]:
response = await claude.arun(
    prompt=prompt,
    model_name="claude-3-5-sonnet-20240620",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up a comfortable party mood for you. Here's what I've done:\n\n1. Turned on the disco ball: This will create a fun, party atmosphere with moving lights.\n2. Started some music: I've chosen energetic music to keep the party mood, but not too loud so it remains comfortable. The tempo is set to 120 BPM, which is a good middle ground for a variety of party music styles.\n3. Dimmed the lights: I've set the brightness to 0.4 (on a scale from 0 to 1), which should create a cozy, intimate atmosphere without being too dark.\n\nThese settings should create a comfortable party mood that's lively but not overwhelming. The spinning disco ball will add a festive touch, the energetic music at a moderate volume will keep the energy up without being too intense, and the dimmed lights will create a relaxed ambiance.\n\nIs there anything else you'd like me to adjust to make the party mood even more comfortable?"}],
  'name': None},

Pydantic tool use

In [50]:
claude = ClaudeFunctionalChat(api_key_env_name="ANTHROPIC_API_KEY")

In [51]:
response = claude.run(
    prompt=prompt,
    model_name="claude-3-5-sonnet-20240620",
    tools=tools_pydantic,
    tool_configs=tool_configs_pydantic,
)
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up a comfortable party mood for you. Here's what I've done:\n\n1. Powered on the disco ball: The spinning disco ball will add a fun, party atmosphere to the room.\n\n2. Started the music: I've chosen music that is energetic but not too loud, with a moderate tempo of 120 BPM. This should create a lively atmosphere without being overwhelming.\n\n3. Dimmed the lights: I've set the brightness to 0.4 (on a scale of 0 to 1), which should provide a cozy, intimate lighting level that's perfect for a comfortable party mood.\n\nThese settings should create a welcoming and enjoyable atmosphere for your party. The combination of the spinning disco ball, energetic yet not too loud music, and dimmed lighting should make for a comfortable and fun environment.\n\nIs there anything else you'd like me to adjust to perfect the party mood?"}],
  'name': None},
 'usage': {'model_name': 'claude-3-5-sonnet-20240620',
  'prompt_tokens':

### Claude on Amazon Bedrock

In [52]:
claude_bedrock = ClaudeFunctionalChat(
    api_type="bedrock",
    aws_access_key_env_name="AWS_ACCESS_KEY",
    aws_secret_key_env_name="AWS_SECRET_KEY",
    aws_region_env_name="AWS_REGION",
)

In [53]:
response = claude_bedrock.run(
    prompt=prompt,
    model_name="anthropic.claude-3-sonnet-20240229-v1:0",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

response.model_dump()

{'message': {'role': 'assistant', 'content': [], 'name': None},
 'usage': {'model_name': 'anthropic.claude-3-sonnet-20240229-v1:0',
  'prompt_tokens': 989,
  'completion_tokens': 159},
 'prompt': [{'role': 'user',
   'content': [{'text': 'Turn this place into a comfortable party mood!',
     'type': 'text'}]},
  {'role': 'assistant',
   'content': [{'text': "Okay, let's set the mood for a fun party!",
     'type': 'text'},
    {'id': 'toolu_01DrM9NNyvbp3P6veqYaHmde',
     'type': 'tool_use',
     'input': {'power': True},
     'name': 'power_disco_ball'}]},
  {'role': 'user',
   'content': [{'tool_use_id': 'toolu_01DrM9NNyvbp3P6veqYaHmde',
     'type': 'tool_result',
     'content': 'Disco ball is spinning!'}]}]}

In [54]:
repsonse = await claude_bedrock.arun(
    prompt=prompt,
    model_name="anthropic.claude-3-sonnet-20240229-v1:0",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)
response.model_dump()

{'message': {'role': 'assistant', 'content': [], 'name': None},
 'usage': {'model_name': 'anthropic.claude-3-sonnet-20240229-v1:0',
  'prompt_tokens': 989,
  'completion_tokens': 159},
 'prompt': [{'role': 'user',
   'content': [{'text': 'Turn this place into a comfortable party mood!',
     'type': 'text'}]},
  {'role': 'assistant',
   'content': [{'text': "Okay, let's set the mood for a fun party!",
     'type': 'text'},
    {'id': 'toolu_01DrM9NNyvbp3P6veqYaHmde',
     'type': 'tool_use',
     'input': {'power': True},
     'name': 'power_disco_ball'}]},
  {'role': 'user',
   'content': [{'tool_use_id': 'toolu_01DrM9NNyvbp3P6veqYaHmde',
     'type': 'tool_result',
     'content': 'Disco ball is spinning!'}]}]}

Pydantic tool use

In [55]:
claude_bedrock = ClaudeFunctionalChat(
    api_type="bedrock",
    aws_access_key_env_name="AWS_ACCESS_KEY",
    aws_secret_key_env_name="AWS_SECRET_KEY",
    aws_region_env_name="AWS_REGION",
)

In [56]:
response = claude_bedrock.run(
    prompt=prompt,
    model_name="anthropic.claude-3-sonnet-20240229-v1:0",
    tools=tools_pydantic,
    tool_configs=tool_configs_pydantic,
)
response.model_dump()

{'message': {'role': 'assistant', 'content': [], 'name': None},
 'usage': {'model_name': 'anthropic.claude-3-sonnet-20240229-v1:0',
  'prompt_tokens': 986,
  'completion_tokens': 157},
 'prompt': [{'role': 'user',
   'content': [{'text': 'Turn this place into a comfortable party mood!',
     'type': 'text'}]},
  {'role': 'assistant',
   'content': [{'text': "Okay, let's set the mood for a fun party!",
     'type': 'text'},
    {'id': 'toolu_01EgSqTWWKcQxCTKk9ij2S4f',
     'type': 'tool_use',
     'input': {'power': True},
     'name': 'PowerDiscoBall'}]},
  {'role': 'user',
   'content': [{'tool_use_id': 'toolu_01EgSqTWWKcQxCTKk9ij2S4f',
     'type': 'tool_result',
     'content': 'Disco ball is spinning!'}]}]}

# Universal tool config system

Tool config the user defines is common for various llm. It is converted to client tools in clinet's FunctionCallingModule before API call. Let me show you the process of conversion.

In [57]:
from langrila.openai import OpenAIToolConfig

converted_configs = OpenAIToolConfig.from_universal_configs(tool_configs)
converted_configs

[OpenAIToolConfig(name='power_disco_ball', description='Powers the spinning disco ball.', parameters=OpenAIToolParameter(type='object', properties=[OpenAIToolProperty(name='power', type='boolean', description='Boolean to spin disco ball.', enum=None, items=None)], required=['power']), strict=None, type='function'),
 OpenAIToolConfig(name='start_music', description='Play some music matching the specified parameters.', parameters=OpenAIToolParameter(type='object', properties=[OpenAIToolProperty(name='energetic', type='boolean', description='Whether the music is energetic or not.', enum=None, items=None), OpenAIToolProperty(name='loud', type='boolean', description='Whether the music is loud or not.', enum=None, items=None), OpenAIToolProperty(name='bpm', type='string', description='The beats per minute of the music.', enum=['60', '120', '180'], items=None)], required=['energetic', 'loud', 'bpm']), strict=None, type='function'),
 OpenAIToolConfig(name='dim_lights', description='Dim the lig

Then these client tool config objects is transformed to client tools by format() method.

In [58]:
converted_configs[0].format()

{'type': 'function',
 'function': {'name': 'power_disco_ball',
  'description': 'Powers the spinning disco ball.',
  'parameters': {'type': 'object',
   'required': ['power'],
   'properties': {'power': {'type': 'boolean',
     'description': 'Boolean to spin disco ball.'}}}}}

Other client like Gemini and Claude is the same interface, so you could reuse tool_configs you initially defines.

In [59]:
from langrila.gemini.genai import GeminiToolConfig

In [60]:
converted_configs = GeminiToolConfig.from_universal_configs(tool_configs)
converted_configs

[GeminiToolConfig(name='power_disco_ball', description='Powers the spinning disco ball.', parameters=GeminiToolParameter(type='object', properties=[GeminiToolProperty(name='power', type='boolean', description='Boolean to spin disco ball.', enum=None, items=None)], required=['power']), strict=None),
 GeminiToolConfig(name='start_music', description='Play some music matching the specified parameters.', parameters=GeminiToolParameter(type='object', properties=[GeminiToolProperty(name='energetic', type='boolean', description='Whether the music is energetic or not.', enum=None, items=None), GeminiToolProperty(name='loud', type='boolean', description='Whether the music is loud or not.', enum=None, items=None), GeminiToolProperty(name='bpm', type='string', description='The beats per minute of the music.', enum=['60', '120', '180'], items=None)], required=['energetic', 'loud', 'bpm']), strict=None),
 GeminiToolConfig(name='dim_lights', description='Dim the lights.', parameters=GeminiToolParame

In [61]:
converted_configs[0].format()

name: "power_disco_ball"
description: "Powers the spinning disco ball."
parameters {
  type_: OBJECT
  properties {
    key: "power"
    value {
      type_: BOOLEAN
      description: "Boolean to spin disco ball."
    }
  }
  required: "power"
}

# Limitation for the function calling

The output of the tools is required to be serializable when `tool_only` option is False as conversation memory is used for bridging function calling module and chat module internally even if you don't specify conversation memory. So please implement tools to return serializable object if you use tools with no `tool_only` option. If you want to get non-serializable object with tool calling, please specify `tool_only` option. Another way is to use `{Client}FunctionCallingModule`. Here is an example for OpenAI API.

In [62]:
from langrila.openai import OpenAIFunctionCallingModule

openai_function = OpenAIFunctionCallingModule(api_key_env_name="API_KEY")

prompt = "Turn this place into a comfortable party mood!"
response = openai_function.run(
    prompt,
    model_name="gpt-4o-2024-08-06",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [63]:
# {Client}FunctionCallingModule returns FunctionCallingRestuls instance
response

FunctionCallingResults(usage=Usage(prompt_tokens=171, completion_tokens=75, total_tokens=246), results=[Message(role='function', content=[ToolContent(output='Disco ball is spinning!', call_id='call_B0J2WNkWPngP3cWBT6nBXS2w', args='{"power": true}', funcname='power_disco_ball')], name='power_disco_ball'), Message(role='function', content=[ToolContent(output="Starting music! energetic=True loud=True, bpm='120'", call_id='call_ZdTLOMdb7RqkAdMn2IjLHtXC', args='{"energetic": true, "loud": true, "bpm": "120"}', funcname='start_music')], name='start_music'), Message(role='function', content=[ToolContent(output='Lights are now set to 0.5', call_id='call_59KQxxVzRO7DhqnRiSbWGOml', args='{"brightness": 0.5}', funcname='dim_lights')], name='dim_lights')], calls=Message(role='function_call', content=[ToolCall(name='power_disco_ball', args='{"power": true}', call_id='call_B0J2WNkWPngP3cWBT6nBXS2w'), ToolCall(name='start_music', args='{"energetic": true, "loud": true, "bpm": "120"}', call_id='call_Z